In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.stats import pearsonr
import numpy as np

def weight_similarity(model1, model2):
    """Calculate cosine similarity between flattened model weights."""
    weights1 = torch.cat([p.view(-1) for p in model1.parameters()])
    weights2 = torch.cat([p.view(-1) for p in model2.parameters()])
    return F.cosine_similarity(weights1.unsqueeze(0), weights2.unsqueeze(0)).item()

def output_similarity(model1, model2, input_data):
    """Calculate Pearson correlation between model outputs."""
    with torch.no_grad():
        out1 = model1(input_data).numpy().flatten()
        out2 = model2(input_data).numpy().flatten()
    return pearsonr(out1, out2)[0]

def activation_similarity(model1, model2, input_data):
    """Compare activations of each layer."""
    similarities = []
    
    def hook_fn(module, input, output):
        return output
    
    for (name1, module1), (name2, module2) in zip(model1.named_modules(), model2.named_modules()):
        if isinstance(module1, nn.Linear) and isinstance(module2, nn.Linear):
            handle1 = module1.register_forward_hook(hook_fn)
            handle2 = module2.register_forward_hook(hook_fn)
            
            with torch.no_grad():
                out1 = model1(input_data)
                out2 = model2(input_data)
            
            act1 = module1.output.numpy().flatten()
            act2 = module2.output.numpy().flatten()
            
            similarity = pearsonr(act1, act2)[0]
            similarities.append((name1, similarity))
            
            handle1.remove()
            handle2.remove()
    
    return similarities

def compare_models(model1, model2, input_data):
    """Compare two models using multiple similarity metrics."""
    weight_sim = weight_similarity(model1, model2)
    output_sim = output_similarity(model1, model2, input_data)
    activation_sims = activation_similarity(model1, model2, input_data)
    
    print(f"Weight Similarity: {weight_sim:.4f}")
    print(f"Output Similarity: {output_sim:.4f}")
    print("Activation Similarities:")
    for name, sim in activation_sims:
        print(f"  {name}: {sim:.4f}")

# Example usage
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create two models
model1 = SimpleNet()
model2 = SimpleNet()

# Generate some random input data
input_data = torch.randn(100, 10)

# Compare the models
# compare_models(model1, model2, input_data)

In [4]:
import main_mfrl_REINFORCE_RA2C as RA2C
from mfrl_lib.lib import *
import torch

agents = [RA2C.Agent(topology, i) for i in range(node_n)]
for i in range(node_n):
    agents[i].pinet.load_state_dict(torch.load(f"models/RA2C_agent_{i}.pth", map_location=device))

In [11]:
agents[0].pinet((torch.Tensor([[[0.5, 0.5, 0.0]]])).to(device),
                torch.zeros(1, 1, 32).to(device),
                torch.zeros(1, 1, 32).to(device))

(tensor([[[0.0160, 0.0866]]], device='cuda:0', grad_fn=<ViewBackward0>),
 tensor([[[0.4286]]], device='cuda:0', grad_fn=<ViewBackward0>),
 tensor([[[-0.1347,  0.1667,  0.0982,  0.0410,  0.1112, -0.1272, -0.1506,
           -0.0202, -0.0423, -0.1044,  0.0402,  0.0169,  0.0850, -0.0426,
            0.0573, -0.0425, -0.0681,  0.0520, -0.0306,  0.1004,  0.0918,
            0.1633,  0.0376, -0.0949, -0.0646, -0.1189,  0.0903, -0.0103,
           -0.0680,  0.1779,  0.0561,  0.0513]]], device='cuda:0',
        grad_fn=<CudnnRnnBackward0>),
 tensor([[[-0.2378,  0.2745,  0.1753,  0.0765,  0.1931, -0.2018, -0.2510,
           -0.0386, -0.0791, -0.1923,  0.0746,  0.0291,  0.1457, -0.0921,
            0.1103, -0.0966, -0.1212,  0.0981, -0.0668,  0.1725,  0.2007,
            0.2732,  0.0708, -0.1552, -0.1359, -0.2274,  0.1484, -0.0207,
           -0.1142,  0.2966,  0.1233,  0.0876]]], device='cuda:0',
        grad_fn=<CudnnRnnBackward0>))

In [13]:
output_similarity(agents[0].pinet, agents[7].pinet, ((torch.Tensor([[[0.5, 0.5, 0.0]]])).to(device),
                                                     torch.zeros(1, 1, 32).to(device),
                                                     torch.zeros(1, 1, 32).to(device)))

TypeError: Pinet.forward() missing 2 required positional arguments: 'h' and 'c'